In [1]:
# import packages
import pandas as pd
import numpy as np
import plotly.express as px
import os

In [2]:
# define desired directory path
directory_path = "/Users/treypallace/Desktop/Fourth Year Fall 2023/ds 4002/Project_Data/linkedin"

# use os.chdir() to change the working directory
os.chdir(directory_path)

# read in the data
job_postings_df = pd.read_csv("job_postings.csv")
job_skills_df = pd.read_csv("job_skills.csv")


### Join the job_postings_df with the job_skills_df

In [3]:
# merge data
df = job_postings_df.merge(job_skills_df, on='job_id',  how='left')

In [4]:
# select columns
df = df[['job_id', 'description', 'max_salary', 'med_salary', 'min_salary', 'pay_period', 'currency', 'work_type','compensation_type','skill_abr', 'location', 'title']]

In [5]:
# convert every salary to hourly pay period 
def convert_to_hourly(row):
    if row['pay_period'] == 'YEARLY':
        if row['work_type'] in ['FULL_TIME', 'CONTRACT']:
            return row['min_salary'] / (40 * 52), row['max_salary'] / (40 * 52)
        elif row['work_type'] == 'PART_TIME':
            return row['min_salary'] / (20 * 52), row['max_salary'] / (20 * 52)
    elif row['pay_period'] == 'MONTHLY':
        if row['work_type'] in ['FULL_TIME', 'CONTRACT']:
            return row['min_salary'] * 12 / (40 * 52), row['max_salary'] * 12 / (40 * 52)
        elif row['work_type'] == 'PART_TIME':
            return row['min_salary'] * 12 / (20 * 52), row['max_salary'] * 12 / (20 * 52)
    else:
        return row['min_salary'], row['max_salary']

# Apply the function to the DataFrame using a lambda function
df[['min_salary', 'max_salary']] = df.apply(lambda row: convert_to_hourly(row), axis=1, result_type='expand')


In [6]:
# calculate average salary
df['avg_salary'] = (df['max_salary'] + df['min_salary']) / 2

In [7]:
nan_counts = df.isna().sum()
nan_counts

job_id                   0
description              2
max_salary           17903
med_salary           25826
min_salary           17903
pay_period           16069
currency             16069
work_type                0
compensation_type    16069
skill_abr              637
location                 0
title                    0
avg_salary           17903
dtype: int64

In [8]:
# filter to remove rows without descriptions or avg_salary
df = df.dropna(subset=['description', 'avg_salary'])

## Model Building: Frequency Count 

In [9]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize the WordNet lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to tokenize a sentence into words
def sentence_to_wordlist(raw_sentence):
    clean_sentence = re.sub("[^a-zA-Z0-9]", " ", raw_sentence)
    tokens = nltk.word_tokenize(clean_sentence)
    return tokens

# Tokenize a description into a list of word lists and remove stopwords
def tokenize(description):
    stripped_description = description.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(stripped_description)
    
    tokenized_sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            tokens = sentence_to_wordlist(raw_sentence)
            tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
            tokenized_sentences.append(tokens)
    
    return tokenized_sentences

# Get Bag of Words (BOW) counts
def get_count_vectors(descriptions):
    vectorizer = CountVectorizer(max_features=10000, ngram_range=(1, 3), stop_words='english')
    count_vectors = vectorizer.fit_transform(descriptions)
    feature_names = vectorizer.get_feature_names_out()
    return feature_names, count_vectors

# Assuming you have a DataFrame 'df' with a 'description' column
feature_names_cv, count_vectors = get_count_vectors(df['description'])


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/treypallace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/treypallace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/treypallace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Most Frequent Words before Extracting for Skills 

In [10]:
# Sum the counts of each word across all documents
word_counts = count_vectors.sum(axis=0)

# Convert the word counts to a list of (word, count) tuples
word_counts_tuples = [(word, count) for word, count in zip(feature_names_cv, word_counts.tolist()[0])]

# Sort the tuples by count in descending order
sorted_word_counts = sorted(word_counts_tuples, key=lambda x: x[1], reverse=True)

# Get the top N most frequent words (adjust N as needed)
top_N_words = 30  # Change this value to get a different number of top words
top_words = sorted_word_counts[:top_N_words]

# Print the top words with their counts
for word, count in top_words:
    print(f'{word}: {count}')


experience: 35388
work: 29388
team: 22440
skills: 17717
business: 17483
including: 16687
management: 14778
ability: 14377
job: 13689
position: 13343
benefits: 12549
time: 12439
company: 12306
years: 11944
support: 11742
required: 11501
data: 11372
role: 11211
new: 10669
development: 10483
customer: 10374
sales: 10043
knowledge: 9746
opportunity: 9675
working: 9610
project: 9339
information: 9179
range: 9096
related: 9009
status: 8561


## Extract for Skills - Named Entity Recognition 


In [12]:
 #!pip install skillNer
 #!python -m spacy download en_core_web_lg

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 3.7 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.7/339.7 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 784.9/784.9 kB 17.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.1/491.1 kB 11.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 5.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [11]:
# imports
import spacy
import en_core_web_lg
from spacy.matcher import PhraseMatcher

# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor
from skillNer.general_params import SKILL_DB

# init params of skill extractor
nlp = en_core_web_lg.load()
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [12]:
#tests a random job description
import random
job_description = df['description'].sample().values[0]

# Extract skills 
annotations = skill_extractor.annotate(job_description)

skill_extractor.describe(annotations)

/Users/treypallace/miniconda3/lib/python3.10/site-packages/skillNer/utils.py:99: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  vec_similarity = token1.similarity(token2)


In [14]:
# Extract values from 'doc_node_value' and save them into a list
values_list = []
for item in annotations['results']['full_matches']:
    values_list.append(item['doc_node_value'])
    
# remove repeats
skills_list = list(set(values_list))

print(skills_list)

['operating system',
 'device management',
 'mobile device',
 'mobile device management',
 'device management',
 'application packaging',
 'production system',
 'directory service',
 'technical documentation',
 'functional specification',
 'technical support',
 'office 365 exchange online',
 'user account',
 'network management',
 'active directory',
 'user directory',
 'directory structure',
 'group policy',
 'functional specification']

In [26]:
## ****takes a VERY long time to run, just import the extracted_skills.csv dataframe**** 
def extract_skills(description, row_index):
    try:
        # Extract skills
        annotations = skill_extractor.annotate(description)

        # Extract values from 'doc_node_value' and save them into a list
        values_list = []
        for item in annotations['results']['full_matches']:
            values_list.append(item['doc_node_value'])

        # Remove duplicates
        skills_list = list(set(values_list))

        return skills_list
    except Exception as e:
        # Handle any exceptions (e.g., if the extraction process fails)
        print(f"Error extracting skills from description at row {row_index}: {e}")
        return []

In [33]:
df4['skills_extracted'] = df4.apply(lambda row: extract_skills(row['description'], row.name), axis=1)

/Users/treypallace/miniconda3/lib/python3.10/site-packages/skillNer/utils.py:99: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  vec_similarity = token1.similarity(token2)


Error extracting skills from description at row 3: list index out of range
Error extracting skills from description at row 4: list index out of range
Error extracting skills from description at row 24: list index out of range
Error extracting skills from description at row 25: list index out of range
Error extracting skills from description at row 29: list index out of range
Error extracting skills from description at row 30: list index out of range
Error extracting skills from description at row 31: list index out of range
Error extracting skills from description at row 34: 'bonus' is not in list
Error extracting skills from description at row 35: 'bonus' is not in list
Error extracting skills from description at row 39: list index out of range
Error extracting skills from description at row 40: list index out of range
Error extracting skills from description at row 41: list index out of range
Error extracting skills from description at row 42: list index out of range
Error extracting

Error extracting skills from description at row 1964: list index out of range
Error extracting skills from description at row 2008: list index out of range
Error extracting skills from description at row 2009: list index out of range
Error extracting skills from description at row 2010: 'build' is not in list
Error extracting skills from description at row 2011: 'build' is not in list
Error extracting skills from description at row 2013: 'lead' is not in list
Error extracting skills from description at row 2014: 'lead' is not in list
Error extracting skills from description at row 2015: 'build' is not in list
Error extracting skills from description at row 2016: 'build' is not in list
Error extracting skills from description at row 2017: 'build' is not in list
Error extracting skills from description at row 2024: 'take' is not in list
Error extracting skills from description at row 2025: 'take' is not in list
Error extracting skills from description at row 2035: 'build' is not in list


In [ ]:
# takes a VERY LONG time to run. Just import dataset 
df['skills_extracted'] = df.apply(lambda row: extract_skills(row['description'], row.name), axis=1)

In [36]:
empty_skills_df = skills_df[skills_df['skills_extracted'].apply(len) == 0].copy()

In [37]:
final_df = df[df['skills_extracted'].apply(len) > 0]

In [40]:
final_df.to_csv('extracted_skills.csv', index=False)

## Most Frequent Skills after NER

In [43]:
# flatten the lists of skills into a Series
flattened_skills = final_df['skills_extracted'].explode()

# get word frequencies
word_counts = flattened_skills.value_counts().reset_index()

# rename the columns 
word_counts.columns = ['Word', 'Frequency']

# print the top N most frequent words (e.g., top 10)
top_N = 30
print(word_counts.head(top_N))


                           Word  Frequency
0                           com       2243
1                 problem solve       1557
2              customer service       1540
3               job description       1275
4              microsoft office       1115
5            project management       1030
6           write communication       1013
7                         reach        709
8          organizational skill        708
9                           San        703
10             computer science        679
11              time management        677
12                          Act        543
13   verbal communication skill        481
14          process improvement        466
15                detail orient        457
16                 self starter        449
17          customer experience        442
18             employee benefit        420
19                          act        416
20             analytical skill        413
21                 supply chain        408
22         